# **Multi gpu - CHPS0904 - Multi GPU Programming Models for HPC and AI**

# Partie 1 : Version CPU 

- Jacobi séquentiel sur CPU (pas de parallélisme, pas de MPI, pas de GPU)
- Lecture des paramètres depuis la ligne de commande : nx, ny, max_iter, tol, affichage
- Allocation dynamique des deux tableaux (a, a_new) pour stocker la grille
- Initialisation des tableaux avec conditions aux bords : gauche/droite à 1, intérieur à 0
- Boucle d'itération Jacobi utilisant une fonction dédiée (jacobi_step)
- Application d'une condition périodique sur les bords haut/bas (apply_periodic_bc)
- Calcul du maximum des différences (norme d’erreur) entre a et a_new pour tester la convergence
- Échange des pointeurs a et a_new pour alterner à chaque itération sans recopie de données
- Affichage optionnel de la grille à la fin, plus affichage des valeurs aux coins et au centre
- Impression du nombre d’itérations, de l’erreur finale, du temps d’exécution total (avec omp_get_wtime, mais sans parallélisme)
- Découpage en fonctions indépendantes pour chaque étape (initialisation, étape Jacobi, conditions bords, affichage, etc.)


## Compilation 

In [59]:
%%bash
cd CPU
make clean 
make


rm -f jacobi_seq
gcc -O3 -Wall -march=native -funroll-loops -ffast-math -std=c99 -fopenmp -o jacobi_seq jacobi.c laplace2d.c -lm


## Execution

In [8]:
%%bash 
./CPU/jacobi_seq 4096 4096 1000 1e-6 0

Jacobi solver stopped after 1000 iterations
Reached max_iter = 1000 (error = 2.419263e-04 > tol = 1.000000e-06)
Elapsed time   : 15.544706 seconds
Values at corners: [0,0]=1.00  [0,4095]=1.00
Value at center [2048,2048] = 0.000000


# Partie 2 : Version CPU + GPU 

- Ajout du GPU avec CUDA : calculs réalisés sur le device, et non plus sur le CPU.
- Utilisation de kernels CUDA pour :
    - Effectuer Jacobi sur le tableau (jacobi_step).
    - Appliquer les conditions aux limites périodiques en Y (apply_periodic_bc).
- Allocation mémoire séparée sur le CPU (host) et le GPU (device).
- Copie initiale des données du host vers le device avant le calcul.
- Boucle principale :
    - Lancement du kernel Jacobi sur la grille (d_a_new ← d_a).
    - Lancement du kernel d’application des bords périodiques en Y.
    - Calcul de l’erreur de convergence directement sur le GPU avec **Thrust** :  
      Thrust est une bibliothèque C++ intégrée à CUDA qui fournit des primitives algorithmiques (comme les réductions, tris, scans) optimisées pour le GPU. Ici, j'utilise `thrust::transform_reduce` pour parcourir les tableaux sur le GPU et calculer en parallèle l’erreur maximale entre deux itérations, sans repasser les données sur le CPU.
    - Inversion des pointeurs device (std::swap) pour éviter une recopie à chaque itération.
- Mesure du temps CUDA (calcul pur) via cudaEvent_t, en plus du temps global CPU.
- Récupération des résultats finaux du device vers le host pour affichage.
- Affichage du temps CUDA, du temps global, du nombre d’itérations, de l’erreur finale, et de l’état final de la grille si demandé.


## Compilation 

In [11]:
%%bash
cd CPU-GPU
make

nvcc -O3 -arch=sm_60 -std=c++11        -o jacobi_cuda jacobi.cu -lm                             


## Execution

In [13]:
%%bash 
./CPU-GPU/jacobi_cuda 4096 4096 1000 1e-6 0

Solveur Jacobi CUDA convergé en 1000 itérations (erreur = 2.419e-04)
Temps passé (calcul CUDA) : 0.238161 s
Temps total du programme (tout inclus) : 2.551201 s


# Partie 3 : MPI + GPUs

- Ajout de la **parallelisation distribuée** avec **MPI** : le domaine 2D est découpé entre plusieurs processus MPI, chacun pouvant s’exécuter sur un GPU distinct.
- Allocation mémoire device par sous-domaine local (chaque rang alloue uniquement sa bande locale + halos).
- Chaque processus traite une bande de la grille ; gestion fine des indices pour découpage non uniforme si nécessaire.
- Ajout d’une communication explicite entre processus MPI à chaque itération pour échanger les **halos** (les halos sont les bandes de points supplémentaires ajoutées au bord de chaque sous-domaine, servant à stocker temporairement les valeurs des points voisins d’un autre sous-domaine ; ils permettent à chaque processus de calculer ses points de frontière sans accès direct à toute la grille globale) via `MPI_Sendrecv`.
- Les échanges de halos utilisent les pointeurs device CUDA : le code est compatible MPI CUDA-aware, c’est-à-dire que les transferts MPI sont réalisés directement depuis/vers la mémoire GPU.
- Synchronisation inter-rangs avant/après le calcul (MPI_Barrier) pour bien délimiter les phases de calcul scientifique dans les mesures de temps.
- Réduction MPI (`MPI_Allreduce`) pour calculer l’erreur globale de convergence sur l’ensemble des sous-domaines (chaque GPU calcule l’erreur locale avec Thrust, puis réduction MPI).
- Regroupement et affichage des résultats sur le rang 0 via des communications MPI pour rassembler toute la grille.


## Compilation

In [3]:
%%bash
cd MPI-GPU
make nsight



mkdir -p nsight
# Adapte la ligne ci-dessous selon les arguments à donner à ton programme
nsys profile -o nsight/jacobi_cuda_profile \
	mpirun -np 4 ./jacobi_cuda 4096 4096 10000 1e-6 0
Solveur Jacobi MPI+multiGPU convergé en 10000 itérations (erreur = 2.420e-05)
Temps calcul MPI+multiGPU (hors init): 10.474947 s
Temps total du programme (allocs, MPI, init, calcul, etc.): 12.065351 s
Generating '/tmp/nsys-report-35d9.qdstrm'
[1/1] [========================100%] jacobi_cuda_profile.nsys-rep
Generated:
    /gpfs/home/scortinhal/CHPS0904/MultiGPU/MPI-GPU/nsight/jacobi_cuda_profile.nsys-rep


In [20]:
%%bash
cd MPI-GPU

mpirun -np 4 ./jacobi_cuda 4096 4096 1000 1e-6 0



Solveur Jacobi MPI+multiGPU convergé en 1000 itérations (erreur = 2.422e-04)
Temps calcul MPI+multiGPU (hors init): 1.312076 s
Temps total du programme (allocs, MPI, init, calcul, etc.): 2.197987 s


# Partie 4 : MPI + Overlap

- Modif pour  **asynchrone et overlap** entre les communications MPI (transferts des halos) et les calculs sur le GPU.
- Utilisation de **plusieurs streams CUDA** :
  - Un stream pour chaque bande (top, bottom, intérieur) afin de lancer simultanément calculs et transferts.
  - Un stream dédié aux copies mémoire host/device (cudaMemcpyAsync) pour les halos.
- Implémentation du **schéma overlap** :
  - Les halos (zones tampon en haut et en bas du sous-domaine, servant à stocker les valeurs échangées avec les voisins MPI et nécessaires pour calculer les points en bordure) sont d’abord transférés du GPU vers le CPU de façon asynchrone.
  - Pendant que les échanges MPI asynchrones (MPI_Isend/Irecv) des halos sont en cours, le calcul sur la bande intérieure (qui ne dépend pas des halos) est lancé.
  - Dès que les halos sont reçus, ils sont recopiés du CPU vers le GPU (toujours en asynchrone) ; on attend la fin de ce transfert avant de calculer les bandes du haut et du bas, dépendantes des halos.
- Synchronisation sur les streams CUDA pour garantir la cohérence des calculs (on synchronise seulement ce qui est nécessaire, pas tout le GPU à chaque étape).
- Attente asynchrone des communications MPI pour maximiser l’overlap (on lance la réduction Thrust/Allreduce seulement quand tout est fini).
- La convergence reste mesurée avec **Thrust** (comme avant : `transform_reduce` calcule directement sur le GPU le maximum des différences entre deux itérations, sans repasser par le CPU).
- Ce schéma permet d’**overlap** le calcul et la communication


In [4]:
%%bash
cd MPI-GPU-overlap
make nsight


mkdir -p nsight
# Adapte la ligne ci-dessous selon les arguments à donner à ton programme
nsys profile -o nsight/jacobi_cuda_profile \
	mpirun -np 4 ./jacobi_cuda 4096 4096 10000 1e-6 0
Solveur Jacobi MPI+multiGPU (full overlap memcpyasync) convergé en 10000 itérations (error = 2.420e-05)
Temps calcul MPI+multiGPU (hors alloc/init): 10.920290 s
Temps total du programme (tout inclus): 12.466443 s
Generating '/tmp/nsys-report-b766.qdstrm'
[1/1] [========================100%] jacobi_cuda_profile.nsys-rep
Generated:
    /gpfs/home/scortinhal/CHPS0904/MultiGPU/MPI-GPU-overlap/nsight/jacobi_cuda_profile.nsys-rep


In [23]:
%%bash

cd MPI-GPU-overlap
mpirun -np 4 ./jacobi_cuda 4096 4096 1000 1e-6 0

Solveur Jacobi MPI+multiGPU (full overlap memcpyasync) convergé en 1000 itérations (error = 2.422e-04)
Temps calcul MPI+multiGPU (hors alloc/init): 1.110611 s
Temps total du programme (tout inclus): 1.979046 s


# Partie 5 : NCCL


- Utilisation de **NCCL** (NVIDIA Collective Communication Library) pour les communications entre GPUs, à la place de MPI pur ou CUDA-aware MPI.
    - Les échanges de halos (zones tampon haut/bas entre sous-domaines voisins) se font ici directement entre GPUs via `ncclSend` et `ncclRecv`, de façon totalement device-to-device, sans passer par la RAM du CPU.
    - Le schéma d’échange est `ncclGroupStart/ncclGroupEnd` pour lancer plusieurs envois/réceptions en une seule opération collective, ce qui améliore le débit.
- Initialisation de NCCL : un communicateur NCCL est créé pour permettre les échanges collectifs entre tous les GPUs.
- Synchronisation sur un **stream CUDA** après les échanges de halos NCCL, pour garantir que les données sont prêtes avant le calcul.
- Le calcul local du Jacobi reste inchangé : il est fait sur le GPU via un kernel CUDA.
- La mesure de la convergence utilise toujours **Thrust** pour faire le calcul de la norme maximale sur GPU.
- La réduction finale de l’erreur (convergence globale) se fait par **MPI_Allreduce** : NCCL n’est utilisé que pour les échanges de halos, la convergence reste synchronisée au niveau MPI.
- Les chronos sont placés pour mesurer :
    - Le temps global (du début à la fin du programme).
    - Le temps après initialisation de NCCL.
    - Le temps pur de la boucle Jacobi.
- L'affichage/réassemblage final des résultats reste identique aux versions précédentes.


## Compilation et execution

In [62]:
%%bash
cd NCCL

eval $(spack load --sh nccl)
export LD_LIBRARY_PATH=$(spack location -i nccl)/lib:$LD_LIBRARY_PATH

make
mpirun -np 4 ./jacobi_nccl 4096 4096 1000 1e-6 0


make: Nothing to be done for 'all'.
Solveur Jacobi NCCL convergé en 1000 itérations (error = 2.422e-04)
1. Temps total du programme (MPI_Init → fin)                : 6.495399 s
2. Temps après init NCCL (juste avant alloc/init CUDA)      : 1.747198 s
3. Temps NCCL (calcul pur boucle Jacobi)                    : 1.744446 s


# Partie 6 : NCCL - Overlap

- Utilisation de **NCCL** pour l’échange des halos entre GPUs, comme dans la version précédente, mais :
    - On applique un **overlap** entre communications et calcul :
        - Les échanges de halos (lignes du haut et du bas de chaque sous-domaine, nécessaires pour la continuité de la solution) se font via `ncclSend`/`ncclRecv` sur deux streams CUDA distincts (`stream_halo_top`, `stream_halo_bot`).
        - Pendant que les transferts NCCL s’exécutent, on lance immédiatement le calcul Jacobi :
            - Sur les bandes extrêmes du sous-domaine (`iy = 1` pour le haut, `iy = local_ny` pour le bas) sur les mêmes streams que les communications associées.
            - Sur l’intérieur du domaine (lignes qui ne dépendent pas des halos) sur un troisième stream CUDA (`stream_interior`).
    - On effectue une synchronisation (`cudaStreamSynchronize`) sur chaque stream avant de poursuivre, pour garantir que calculs et transferts sont bien terminés.
- Ce **recouvrement comm/calcul** (overlap) permet de gagner du temps si le coût des transferts et du calcul sont similaires ou si l’un des deux prend plus de temps que l’autre.
- Le découpage en bandes (slice) pour les kernels Jacobi permet ce parallélisme : on peut commencer à travailler sur ce qui est prêt sans attendre toute la communication.
- Calcul de la convergence toujours via Thrust sur GPU, puis synchronisation globale via `MPI_Allreduce`.
- Affichage et réassemblage final identiques aux versions précédentes.
- Multiples chronomètres pour séparer le temps total, le temps d’allocation/init CUDA/NCCL, et le temps pur de la boucle Jacobi avec overlap.


In [63]:
%%bash
cd NCCL-overlap

eval $(spack load --sh nccl)
export LD_LIBRARY_PATH=$(spack location -i nccl)/lib:$LD_LIBRARY_PATH
export NCCL_IB_HCA=mlx5_0         # si tu as Infiniband
export NCCL_IB_DISABLE=0
export NCCL_P2P_DISABLE=1
make
mpirun -np 1 ./jacobi_nccl 4096 4096 1000 1e-6 0


make: Nothing to be done for 'all'.
NCCL Comm Init: 1.053 s
Solveur Jacobi NCCL + overlap convergé en 1000 itérations (error = 2.422e-04)
1. Temps total du programme (MPI_Init → fin)                 : 2.094690 s
2. Temps après init NCCL (juste avant alloc/init CUDA)       : 0.380852 s
3. Temps calcul NCCL + overlap (boucle Jacobi uniquement)    : 0.379018 s


# Partie 7 : NCCL - graph

- Ajout de l'utilisation des **CUDA Graphs** pour la capture et l'exécution de la boucle Jacobi :
    - On capture la séquence "comm/halo + calcul Jacobi (bande intérieures et extrêmes) + swap" en graph CUDA, avec deux variantes :
        - un graph sans calcul d’erreur,
        - un graph qui inclut le calcul d’erreur locale toutes les 10 itérations (ou à une fréquence réglable).
    - Cela permet d’**accélérer les itérations** (moins d’overhead de lancement de kernels, moins de passages CPU/GPU, ordonnancement optimisé par CUDA).
- L'attribution du GPU à chaque processus MPI se fait proprement via le "local_rank" obtenu à partir du communicator partagé MPI (pour que deux processus MPI du même nœud ne prennent pas le même GPU).
- Toujours un découpage en sous-domaines horizontaux équilibré même si ny-2 n'est pas multiple du nombre de processus.
- Toujours overlap entre communications (NCCL) et calcul (multi-streams CUDA), mais cette logique est incluse à l’intérieur du CUDA Graph.
- Nettoyage mémoire : destruction des graphs et exécuteurs (`cudaGraphExecDestroy`) à la fin.
- Pour le reste : gestion des halos (lignes fantômes haut/bas pour l’échange entre voisins), calcul local Jacobi en slice, calcul de l’erreur via Thrust puis réduction MPI pour la convergence globale, impression et réassemblage final.


In [12]:
%%bash
cd NCCL-graph

eval $(spack load --sh nccl)
export LD_LIBRARY_PATH=$(spack location -i nccl)/lib:$LD_LIBRARY_PATH

make
mpirun -np 4 ./jacobi_nccl 4096 4096 1000 1e-6 0


make: Nothing to be done for 'all'.
Solveur Jacobi NCCL+Graphs convergé en 1000 itérations (error = 2.442e-04) en 0.148811s
Error while terminating subprocess (pid=3719123): 


# Partie 8 : NVSHMEM

- Introduction de **NVSHMEM** comme méthode de communication pour les échanges de halos entre GPUs, en remplacement de NCCL ou MPI direct :
    - NVSHMEM permet le "Remote Memory Access" (RMA) : chaque GPU peut écrire directement dans la mémoire d’un voisin (put), sans impliquer le CPU, ce qui réduit la latence et le coût des échanges de halos.
    - Utilisation de `nvshmem_double_put` pour écrire directement la première/dernière ligne réelle dans le halo du voisin.
    - Utilisation de `nvshmem_fence()` pour garantir la visibilité des échanges mémoire, et `nvshmem_barrier_all()` pour synchroniser tous les processus avant de lancer le calcul.
- Initialisation avancée :
    - Calcul et mise à disposition de la taille réelle des buffers (ghost_ny) sur chaque PE pour gérer les halos même si la décomposition est déséquilibrée (i.e., tous les processus n'ont pas le même nombre de lignes).
    - Allocation des tableaux via `nvshmem_malloc` (heap symétrique, identique sur tous les PE).
    - Calcul et configuration dynamique de la taille du heap symétrique avec la variable d’environnement `NVSHMEM_SYMMETRIC_SIZE`.
    - Initialisation de NVSHMEM avec le communicateur MPI pour garantir le mapping entre processus MPI et PE NVSHMEM.
- Toujours le calcul Jacobi en kernel CUDA sur le sous-domaine local, synchronisation CUDA stream comme dans les autres versions.
- Utilisation de **Thrust** pour la réduction GPU de l’erreur max entre a_new et a (calcul de convergence), puis MPI_Allreduce pour la convergence globale.
- Affichage des chronos 
- Rassemblement final et affichage de la grille comme dans les autres versions.


In [8]:
%%bash 
export NVSHMEM_HOME=/apps/2025/manual_install/nvhpc/24.11/Linux_aarch64/24.11/comm_libs/12.6/nvshmem
export CPATH=$NVSHMEM_HOME/include:$CPATH
export LD_LIBRARY_PATH=$NVSHMEM_HOME/lib:$LD_LIBRARY_PATH
export PATH=$NVSHMEM_HOME/bin:$PATH

In [43]:
%%bash

cd NVSHMEM
make clean

rm -f jacobi_nvshmem *.o *.qdrep *.sqlite


In [16]:
%%bash
cd NVSHMEM

# Charge NVSHMEM et NCCL
export NVSHMEM_HOME=/apps/2025/manual_install/nvhpc/24.11/Linux_aarch64/24.11/comm_libs/12.6/nvshmem
export LD_LIBRARY_PATH=$NVSHMEM_HOME/lib:$LD_LIBRARY_PATH
export CPATH=$NVSHMEM_HOME/include:$CPATH
export PATH=$NVSHMEM_HOME/bin:$PATH

eval $(spack load --sh nccl)
export LD_LIBRARY_PATH=$(spack location -i nccl)/lib:$LD_LIBRARY_PATH
export NVSHMEM_ENABLE_ALL_DEVICE_INLINING=1


make 
mpirun -np 4 ./jacobi_nvshmem 4096 4096 1000 1e-6 0


make: Nothing to be done for 'all'.
[NVSHMEM] Temps d'init avant NVSHMEM : 2.531944s
[NVSHMEM] Temps d'init NVSHMEM : 4.115990s
[NVSHMEM] Converged in 1000 iterations | Error: 2.42e-04
Temps de calcul Jacobi (seulement boucle)         : 0.127134s
Temps setup+calcul+affichage (après init SHMEM)   : 0.127302s
Temps total du programme (tout compris)           : 7.639148s


# Partie 9 : NVSHMEM-LTO

- Ajout du support de la compilation **LTO CUDA** (Link-Time Optimization) dans le Makefile :
    - Ajout du flag `-gencode=arch=compute_90,code=lto_90` à la variable NVCCFLAGS pour activer la génération du code intermédiaire LTO pour l’architecture sm_90.
    - Pas de modification du code source, uniquement une optimisation de la génération du binaire final via la chaîne de compilation NVCC pour de meilleures performances potentielles à l’exécution.
    - Optimiser les appels de fonctions entre différents fichiers sources CUDA (inlining, suppression de code mort, etc.)
    - Réorganiser ou fusionner des kernels s’il le juge pertinent.
    - Générer un code plus performant, parfois plus rapide à l’exécution que le même code compilé sans LTO.


In [9]:
%%bash
cd NVSHMEM-LTO


export NVSHMEM_HOME=/apps/2025/manual_install/nvhpc/24.11/Linux_aarch64/24.11/comm_libs/12.6/nvshmem
export CUDA_HOME=/apps/2025/manual_install/nvhpc/24.11/Linux_aarch64/24.11/cuda/12.3
export LD_LIBRARY_PATH=$NVSHMEM_HOME/lib:$CUDA_HOME/lib64:$LD_LIBRARY_PATH
export CPATH=$NVSHMEM_HOME/include:$CUDA_HOME/include:$CPATH
export PATH=$NVSHMEM_HOME/bin:$CUDA_HOME/bin:$PATH

# Charge NCCL via Spack si tu utilises aussi NCCL ailleurs
eval $(spack load --sh nccl)
export LD_LIBRARY_PATH=$(spack location -i nccl)/lib:$LD_LIBRARY_PATH
export NVSHMEM_ENABLE_ALL_DEVICE_INLINING=1

make clean
make
#  lance avec mpirun (remplace nvshmrun si dispo) :
mpirun -np 4 ./jacobi_nvshmem 4096 4096 1000 1e-6 0


rm -f jacobi_nvshmem *.o *.qdrep *.sqlite
nvcc -O3 -arch=sm_90  -std=c++17 --expt-relaxed-constexpr -rdc=true -gencode=arch=compute_90,code=sm_90 -gencode=arch=compute_90,code=lto_90 -Xcompiler "-Wall -Wextra" -ccbin=mpicxx -I/apps/2025/manual_install/nvhpc/24.11/Linux_aarch64/24.11/comm_libs/12.6/nvshmem/include -I/apps/2025/spack_install/linux-rhel9-neoverse_v2/linux-rhel9-neoverse_v2/gcc-11.4.1/cuda-12.6.2-3mzltpzgs4dekx5xvbqzz2no3j3tkcxq/include -o jacobi_nvshmem jacobi.cu -L/apps/2025/manual_install/nvhpc/24.11/Linux_aarch64/24.11/comm_libs/12.6/nvshmem/lib -lnvshmem -lnvshmem_host -L/apps/2025/spack_install/linux-rhel9-neoverse_v2/linux-rhel9-neoverse_v2/gcc-11.4.1/cuda-12.6.2-3mzltpzgs4dekx5xvbqzz2no3j3tkcxq/lib64 -lcudart -lmpi
[NVSHMEM] Temps d'init avant NVSHMEM : 2.511211s
[NVSHMEM] Temps d'init NVSHMEM : 4.076924s
[NVSHMEM] Converged in 1000 iterations | Error: 2.42e-04
Temps de calcul Jacobi (seulement boucle)         : 0.129117s
Temps setup+calcul+affichage (après init SH

# Partie 10 : NVSHMEM-neighborhood_sync-lto


- Utilisation de **NVSHMEM** pour les échanges de halos entre GPUs (Remote Memory Access/RMA) :
    - Chaque processus écrit directement dans la mémoire du halo (bord) de ses voisins grâce à `nvshmem_double_put`.
    - Synchronisation mémoire assurée avec `nvshmem_fence()` puis barrière NVSHMEM pour garantir la cohérence avant le calcul.
- Gestion avancée de la **synchronisation neighborhood** sur device (entre GPU voisins) :
    - Ajout d’un kernel `syncneighborhood_kernel` pour notifier les voisins haut et bas que les halos sont prêts (utilisation de `nvshmemx_signal_op` et `nvshmem_uint64_wait_until_all` pour attendre les notifications des deux côtés).
- Initialisation :
    - Attribution automatique des GPU locaux par rang MPI intra-nœud.
    - Calcul dynamique des tailles de sous-domaines (ghost_ny) pour chaque processus, prise en compte des décompositions déséquilibrées.
    - Allocation de la mémoire sur le heap symétrique NVSHMEM (`nvshmem_malloc`) pour les tableaux et les variables de synchronisation, avec configuration dynamique de la variable d'environnement `NVSHMEM_SYMMETRIC_SIZE` pour garantir un espace mémoire suffisant.
    - Initialisation de NVSHMEM avec un communicateur MPI personnalisé via `nvshmemx_init_attr`.
- Calcul Jacobi toujours exécuté sur GPU via kernel CUDA, synchronisation des streams CUDA.
- Calcul de l’erreur max avec **Thrust** sur GPU (utilisation de la bibliothèque Thrust pour la réduction GPU des différences a_new/a_old), puis MPI_Allreduce pour la convergence globale.
- Mesure détaillée des temps : temps avant et après initialisation NVSHMEM, temps de calcul pur, temps total global, etc.
- Rassemblement final et affichage de la grille sur le rang 0 avec gestion correcte des tailles locales, comme dans les versions précédentes.


In [3]:
%%bash
cd NVSHMEM-neighborhood_sync-lto

# Charge les bonnes librairies NVSHMEM et CUDA
export NVSHMEM_HOME=/apps/2025/manual_install/nvhpc/24.11/Linux_aarch64/24.11/comm_libs/12.6/nvshmem
export CUDA_HOME=/apps/2025/manual_install/nvhpc/24.11/Linux_aarch64/24.11/cuda/12.3
export LD_LIBRARY_PATH=$NVSHMEM_HOME/lib:$CUDA_HOME/lib64:$LD_LIBRARY_PATH
export CPATH=$NVSHMEM_HOME/include:$CUDA_HOME/include:$CPATH
export PATH=$NVSHMEM_HOME/bin:$CUDA_HOME/bin:$PATH

# Charge NCCL via Spack si nécessaire
eval $(spack load --sh nccl)
export LD_LIBRARY_PATH=$(spack location -i nccl)/lib:$LD_LIBRARY_PATH

# Active l'inlining maximal NVSHMEM
export NVSHMEM_ENABLE_ALL_DEVICE_INLINING=1

make clean && make

mpirun -np 4 ./jacobi_nvshmem 4096 4096 1000 1e-6 0




rm -f jacobi_nvshmem *.o *.qdrep *.sqlite
nvcc -O3 -arch=sm_90  -std=c++17 --expt-relaxed-constexpr -rdc=true -gencode=arch=compute_90,code=sm_90 -gencode=arch=compute_90,code=lto_90 -Xcompiler "-Wall -Wextra" -ccbin=mpicxx -I/apps/2025/manual_install/nvhpc/24.11/Linux_aarch64/24.11/comm_libs/12.6/nvshmem/include -I/apps/2025/spack_install/linux-rhel9-neoverse_v2/linux-rhel9-neoverse_v2/gcc-11.4.1/cuda-12.6.2-3mzltpzgs4dekx5xvbqzz2no3j3tkcxq/include -o jacobi_nvshmem jacobi.cu -L/apps/2025/manual_install/nvhpc/24.11/Linux_aarch64/24.11/comm_libs/12.6/nvshmem/lib -lnvshmem -lnvshmem_host -L/apps/2025/spack_install/linux-rhel9-neoverse_v2/linux-rhel9-neoverse_v2/gcc-11.4.1/cuda-12.6.2-3mzltpzgs4dekx5xvbqzz2no3j3tkcxq/lib64 -lcudart -lmpi
[NVSHMEM-neighSync] Converged in 1000 iterations | Error: 2.42e-04
Temps de calcul Jacobi (seulement boucle)         : 0.122526s
Temps setup+calcul+affichage (après init SHMEM)   : 0.122720s
Temps d'init avant NVSHMEM                        : 2.492420s
T

# Partie 11 : nvshmem-norm_overlap-neighborhood_sync+lto 

- le **calcul sur la grille locale est vraiment overlap avec l'échange des halos grâce à un découpage fin des kernels et à la synchronisation asynchrone device-side.

- On découpe la mise à jour Jacobi en :
    - `jacobi_inner_kernel` (zone intérieure, ne dépend pas du halo) qui est lancé pendant que les halos sont envoyés aux voisins.
    - Une fois l'échange de halos terminé (vérifié via le kernel `syncneighborhood_kernel` qui utilise des signaux device NVSHMEMX),  
      on lance `jacobi_border_kernel` sur les deux bandes frontalières haut/bas qui ont besoin du halo à jour.
  Ce découpage permet de **masquer le coût de communication** et d'accélérer la convergence pour des grosses grilles.

- **Synchronisation neighborhood**: au lieu d'une barrière globale (`nvshmem_barrier_all`), chaque processus attend juste un signal de ses deux voisins directs (haut/bas) pour la cohérence des halos : cela limite l'attente et améliore l'overlap comm/calcul.

- Le reste de la structure générale (allocation heap symétrique, gestion des tailles fantômes variables, calcul erreur avec thrust, réduction MPI_Allreduce, réassemblage final) reste identique à la version précédente.

- L'objectif est de **maximiser le recouvrement entre calcul local et communication** des halos, ce qui est particulièrement efficace sur des architectures GPU modernes.



In [14]:
%%bash
cd NVSHMEM-norm_overlap-neighborhood_sync+lto

# Charge les bonnes librairies NVSHMEM et CUDA
export NVSHMEM_HOME=/apps/2025/manual_install/nvhpc/24.11/Linux_aarch64/24.11/comm_libs/12.6/nvshmem
export CUDA_HOME=/apps/2025/manual_install/nvhpc/24.11/Linux_aarch64/24.11/cuda/12.3
export LD_LIBRARY_PATH=$NVSHMEM_HOME/lib:$CUDA_HOME/lib64:$LD_LIBRARY_PATH
export CPATH=$NVSHMEM_HOME/include:$CUDA_HOME/include:$CPATH
export PATH=$NVSHMEM_HOME/bin:$CUDA_HOME/bin:$PATH

# Charge NCCL via Spack si nécessair

eval $(spack load --sh nccl)
export LD_LIBRARY_PATH=$(spack location -i nccl)/lib:$LD_LIBRARY_PATH

# Active l'inlining maximal NVSHMEM
export NVSHMEM_ENABLE_ALL_DEVICE_INLINING=1

make clean && make

mpirun -np 4 ./jacobi_nvshmem 4096 4096 1000 1e-6 0




rm -f jacobi_nvshmem *.o *.qdrep *.sqlite
nvcc -O3 -arch=sm_90  -std=c++17 --expt-relaxed-constexpr -rdc=true -gencode=arch=compute_90,code=sm_90 -gencode=arch=compute_90,code=lto_90 -Xcompiler "-Wall -Wextra" -ccbin=mpicxx -I/apps/2025/manual_install/nvhpc/24.11/Linux_aarch64/24.11/comm_libs/12.6/nvshmem/include -I/apps/2025/spack_install/linux-rhel9-neoverse_v2/linux-rhel9-neoverse_v2/gcc-11.4.1/cuda-12.6.2-3mzltpzgs4dekx5xvbqzz2no3j3tkcxq/include -o jacobi_nvshmem jacobi.cu -L/apps/2025/manual_install/nvhpc/24.11/Linux_aarch64/24.11/comm_libs/12.6/nvshmem/lib -lnvshmem -lnvshmem_host -L/apps/2025/spack_install/linux-rhel9-neoverse_v2/linux-rhel9-neoverse_v2/gcc-11.4.1/cuda-12.6.2-3mzltpzgs4dekx5xvbqzz2no3j3tkcxq/lib64 -lcudart -lmpi
[NVSHMEM-norm_overlap-neighSync+LTO] Converged in 1000 iterations | Error: 2.42e-04
Temps de calcul Jacobi (seulement boucle)         : 0.110126s
Temps setup+calcul+affichage (après init SHMEM)   : 0.110322s
Temps d'init avant NVSHMEM                    

# Partie 12 : nvshmem-norm_overlap-neighborhood_sync+lto[1]  


- **Ici, c'est une version "baseline" (de base, séquentielle côté device)** :
    - On utilise **un seul kernel Jacobi** (`jacobi_kernel_full`) qui fait tout le calcul d'un coup sur tout le sous-domaine local, sans découpage ni overlap calcul/communication.
    - **Pas de découpage** en "zone intérieure" et "zone de bord" : tout est fait d’un bloc, donc la communication des halos ne peut pas être masquée par le calcul local.
    - **Pas de synchronisation neighborhood device** (pas de kernel de synchronisation avec les voisins via NVSHMEMX), ni de stratégie fine pour attendre juste les voisins.
    - On se contente d’un schéma classique : 
        - On fait tout le calcul local,
        - Puis l’erreur est calculée (avec thrust),
        - Puis un `MPI_Allreduce` pour la convergence globale,
        - Et on passe à l’itération suivante.

- **En résumé :**  
  - Version "simple" qui sert de référence pour les perfs : pas d’overlap calcul/comm, pas d’optimisation, pas de synchronisation fine, **juste un Jacobi standard sur tout le domaine local**.
  - Cela permet de comparer l’apport réel des optimisations dans les autres versions (découpage kernels + overlap, synchronisation neighborhood, etc).


In [18]:
%%bash
cd NVSHMEM-norm_overlap-neighborhood_sync+lto[1]

# Charge les bonnes librairies NVSHMEM et CUDA
export NVSHMEM_HOME=/apps/2025/manual_install/nvhpc/24.11/Linux_aarch64/24.11/comm_libs/12.6/nvshmem
export CUDA_HOME=/apps/2025/manual_install/nvhpc/24.11/Linux_aarch64/24.11/cuda/12.3
export LD_LIBRARY_PATH=$NVSHMEM_HOME/lib:$CUDA_HOME/lib64:$LD_LIBRARY_PATH
export CPATH=$NVSHMEM_HOME/include:$CUDA_HOME/include:$CPATH
export PATH=$NVSHMEM_HOME/bin:$CUDA_HOME/bin:$PATH

# Charge NCCL via Spack si nécessair

eval $(spack load --sh nccl)
export LD_LIBRARY_PATH=$(spack location -i nccl)/lib:$LD_LIBRARY_PATH

# Active l'inlining maximal NVSHMEM
export NVSHMEM_ENABLE_ALL_DEVICE_INLINING=1

make clean && make

mpirun -np 4 ./jacobi_nvshmem 4096 4096 1000 1e-6 0




rm -f jacobi_nvshmem *.o *.qdrep *.sqlite
nvcc -O3 -arch=sm_90  -std=c++17 --expt-relaxed-constexpr -rdc=true -gencode=arch=compute_90,code=sm_90 -gencode=arch=compute_90,code=lto_90 -Xcompiler "-Wall -Wextra" -ccbin=mpicxx -I/apps/2025/manual_install/nvhpc/24.11/Linux_aarch64/24.11/comm_libs/12.6/nvshmem/include -I/apps/2025/spack_install/linux-rhel9-neoverse_v2/linux-rhel9-neoverse_v2/gcc-11.4.1/cuda-12.6.2-3mzltpzgs4dekx5xvbqzz2no3j3tkcxq/include -o jacobi_nvshmem jacobi.cu -L/apps/2025/manual_install/nvhpc/24.11/Linux_aarch64/24.11/comm_libs/12.6/nvshmem/lib -lnvshmem -lnvshmem_host -L/apps/2025/spack_install/linux-rhel9-neoverse_v2/linux-rhel9-neoverse_v2/gcc-11.4.1/cuda-12.6.2-3mzltpzgs4dekx5xvbqzz2no3j3tkcxq/lib64 -lcudart -lmpi
[NVSHMEM-baseline-single-rank] Converged in 1000 iterations | Error: 2.42e-04
Temps de calcul Jacobi (seulement boucle)         : 0.087986s
Temps setup+calcul+affichage (après init SHMEM)   : 0.088175s
Temps d'init avant NVSHMEM                        : 